<h1>Table of Contents<span class="tocSkip"></span></h1>
<div class="toc"><ul class="toc-item"></ul></div>

# Construct SVM

Number of features >> observations 

Imports

In [1]:
# SciKit-Optimize:
import skopt
from skopt import gp_minimize, forest_minimize
from skopt.space import Real, Categorical, Integer
from skopt.plots import plot_convergence
from skopt.plots import plot_objective, plot_evaluations
from skopt.utils import use_named_args

# General imports:
import os
import glob
import shutil
import subprocess
import numpy as np
import itertools
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import csv
import seaborn as sns
import matplotlib.cbook
import time
start_time = "Start: "+str(time.ctime())
import warnings
warnings.filterwarnings("ignore",category=matplotlib.cbook.mplDeprecation)
np.seterr(divide='ignore', invalid='ignore')

# Misc. imports:
from sklearn.preprocessing import StandardScaler
from sklearn.ensemble import IsolationForest
from sklearn.metrics import mean_absolute_error
from sklearn.decomposition import PCA
from sklearn.svm import SVR
from scipy import stats
import statistics
import pickle

/home/wwu/anaconda3/envs/freesolv/lib/python3.7/site-packages/sklearn/externals/joblib/__init__.py:15: DeprecationWarning: sklearn.externals.joblib is deprecated in 0.21 and will be removed in 0.23. Please import this functionality directly from joblib, which can be installed with: pip install joblib. If this warning is raised when loading pickled models, you may need to re-serialize those models with scikit-learn 0.21+.
  warnings.warn(msg, category=DeprecationWarning)


Input data

In [2]:
data = '../datasets/train_compiled/dGhydr_train.csv'

MAE and log file

PCA

In [3]:
# construct raw dataset, make sure all values are floats; shuffle rows:
df1 = pd.read_csv(data, index_col='ID')
# collection = collection.apply(pd.to_numeric).astype(float).sample(frac=1)
df1

/home/wwu/anaconda3/envs/freesolv/lib/python3.7/site-packages/IPython/core/interactiveshell.py:3058: DtypeWarning: Columns (272,382,383,384,385,386,387,388,389,390,481,482,483,589,590,591,592,593,594,595,596,597,697,698,699,778,779,794,795,874,875,890,891,920,921,935,948,961,974,987,1000,1013,1026,1089,1115,1123,1124,1131,1139,1140,1188,1531,1532,1533,1534,1555,1556,1605,2021,2049,2050,2052,2080) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


,pfp0,pfp1,pfp2,pfp3,pfp4,pfp5,pfp6,pfp7,pfp8,pfp9,...,AMW,WPath,WPol,Zagreb1,Zagreb2,mZagreb1,mZagreb2,PBF,dGhydr (kcal/mol),uncertainty (kcal/mol)
ID,,,,,,,,,,,,,,,,,,,,,
mobley_5852491,0,0,0,0,0,0,0,0,0,0,...,4.671883,26,2,26.0,27.0,2.111111111111111,1.416667,1.576244e-01,0.19,0.60
mobley_9838013,0,0,0,0,0,0,0,0,0,0,...,4.893823,28,3,24.0,22.0,4.3125,1.375000,1.705255e-01,1.50,0.60
mobley_2410897,0,0,0,0,0,0,0,0,0,0,...,4.505785,4,0,6.0,4.0,2.25,1.000000,8.799730e-02,1.34,0.60
mobley_1893815,0,0,0,0,0,0,0,0,0,0,...,20.736383,28,3,24.0,22.0,4.3125,1.375000,5.848740e-01,1.34,0.10
mobley_2008055,0,0,0,0,0,0,0,0,0,0,...,3.755869,1,0,2.0,1.0,2.0,1.000000,3.361027e-17,0.63,0.60
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
mobley_4149784,0,0,0,0,0,0,1,0,0,0,...,11.453386,414,25,88.0,104.0,4.66667,3.388889,2.726368e-04,0.35,0.13
mobley_6497672,0,0,0,0,0,0,0,0,0,0,...,6.367442,20,2,14.0,12.0,2.75,1.500000,1.585811e-04,0.88,0.60
mobley_9534740,0,0,0,0,0,0,0,0,0,0,...,7.502641,182,19,58.0,68.0,6.05556,2.777778,4.631200e-01,7.38,0.23


In [4]:
def check_dataframe_is_numeric(df):
    'Iterate over all columns and check if numeric.'

    columns_dropped = 0

    for col in df.columns:
        for x in df.loc[:, col]:
            try:
                float(x)
            except ValueError:
#                 print('\'{}\' in {} is not a float.'.format(x, col))
                columns_dropped += 1
                df = df.drop(columns=col)
                break
    
    return df

In [5]:
df2 = check_dataframe_is_numeric(df1)
df2

,pfp0,pfp1,pfp2,pfp3,pfp4,pfp5,pfp6,pfp7,pfp8,pfp9,...,MW,AMW,WPath,WPol,Zagreb1,Zagreb2,mZagreb2,PBF,dGhydr (kcal/mol),uncertainty (kcal/mol)
ID,,,,,,,,,,,,,,,,,,,,,
mobley_5852491,0,0,0,0,0,0,0,0,0,0,...,84.093900,4.671883,26,2,26.0,27.0,1.416667,1.576244e-01,0.19,0.60
mobley_9838013,0,0,0,0,0,0,0,0,0,0,...,88.088815,4.893823,28,3,24.0,22.0,1.375000,1.705255e-01,1.50,0.60
mobley_2410897,0,0,0,0,0,0,0,0,0,0,...,45.057849,4.505785,4,0,6.0,4.0,1.000000,8.799730e-02,1.34,0.60
mobley_1893815,0,0,0,0,0,0,0,0,0,0,...,165.891061,20.736383,28,3,24.0,22.0,1.375000,5.848740e-01,1.34,0.10
mobley_2008055,0,0,0,0,0,0,0,0,0,0,...,30.046950,3.755869,1,0,2.0,1.0,1.000000,3.361027e-17,0.63,0.60
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
mobley_4149784,0,0,0,0,0,0,1,0,0,0,...,251.974485,11.453386,414,25,88.0,104.0,3.388889,2.726368e-04,0.35,0.13
mobley_6497672,0,0,0,0,0,0,0,0,0,0,...,70.041865,6.367442,20,2,14.0,12.0,1.500000,1.585811e-04,0.88,0.60
mobley_9534740,0,0,0,0,0,0,0,0,0,0,...,180.063388,7.502641,182,19,58.0,68.0,2.777778,4.631200e-01,7.38,0.23


In [6]:
collection = df2.apply(pd.to_numeric).astype(float).sample(frac=1)